In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import time
import tqdm
import ujson
import logging
from pathlib import Path
from itertools import islice
from subprocess import check_output
from dask import compute, delayed
import fsspec
from kerchunk.hdf import SingleHdf5ToZarr
import traceback
import ujson
import xarray as xr
import tqdm
import logging
import numpy as np
from pathlib import Path
from itertools import islice
import fsspec
from fsspec.implementations.reference import LazyReferenceMapper
from kerchunk.combine import MultiZarrToZarr
from kerchunk import df

In [3]:
files = ["/Users/sreesanjeevkandasamygokulrajan/Downloads/CLDPROP_D3_VIIRS_SNPP.A2024173.011.2024177004727.nc",
           "/Users/sreesanjeevkandasamygokulrajan/Downloads/CLDPROP_D3_VIIRS_SNPP.A2024174.011.2024178005308.nc" ]

jsons_path = Path("/Users/sreesanjeevkandasamygokulrajan/Desktop/singleJson")

for url in files:
    with fsspec.open(f"{url}") as inf:
        h5chunks = SingleHdf5ToZarr(inf, url, inline_threshold=300)
        json_data = ujson.dumps(h5chunks.translate()).encode()
        outf = f"{url.split('/')[-1]}.json"
        with open(jsons_path / outf, "wb") as f:
            f.write(json_data)

In [4]:
import re
import datetime
def fn_to_time(index, fs, var, fn):
    match = re.search(r'CLDPROP_D3_VIIRS_SNPP\.A(\d{4})(\d{3})\.', fn)
    year = int(match.group(1))
    day_of_year = int(match.group(2))
    return datetime.datetime(year, 1, 1) + datetime.timedelta(days=day_of_year - 1)

In [5]:
parquetPath = f"/Users/sreesanjeevkandasamygokulrajan/Desktop/MultiZarrtoZarr/append.parq"
out = LazyReferenceMapper.create(root=parquetPath)

MultiZarrToZarr(
            ["/Users/sreesanjeevkandasamygokulrajan/Desktop/singleJson/CLDPROP_D3_VIIRS_SNPP.A2024174.011.2024178005308.nc.json"],
            coo_map={'time': fn_to_time},
            coo_dtypes={'time': np.dtype('M8[s]')},
            concat_dims=['time'],
            out=out,
        ).translate()

In [10]:
ds = xr.open_dataset(parquetPath, engine="kerchunk", chunks="auto")
ds

KeyError: 'Cloud_Top_Temperature_Day/.zarray'

In [ ]:
# output = LazyReferenceMapper(root=parquetPath)

out_dict = MultiZarrToZarr.append(
            ["/Users/sreesanjeevkandasamygokulrajan/Desktop/singleJson/CLDPROP_D3_VIIRS_SNPP.A2024173.011.2024177004727.nc.json"],
            original_refs=LazyReferenceMapper(root = "/Users/sreesanjeevkandasamygokulrajan/Desktop/MultiZarrtoZarr/append.parq"),
            coo_map={'time': fn_to_time},
            coo_dtypes={'time': np.dtype('M8[s]')},
            concat_dims=['time'],
            remote_protocol="file",
        )

In [ ]:
ds = xr.open_dataset(parquetPath, engine='kerchunk', chunks="auto")
ds

In [8]:
pip list | grep fsspec

fsspec                    2024.6.1.post8+g3ff5fca.d20240801 /Users/sreesanjeevkandasamygokulrajan/filesystem_spec

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: /Users/sreesanjeevkandasamygokulrajan/.pyenv/versions/3.10.14/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
